# 翻前权益分析

分析常见翻前对局的胜率，并可视化收敛过程。

In [ ]:
from holdem_lab import (
    parse_cards, format_cards,
    EquityRequest, PlayerHand, calculate_equity,
)
import matplotlib.pyplot as plt

## 经典对局

In [ ]:
# 定义经典翻前对局
matchups = [
    ("AA vs KK", "Ah Ad", "Kh Kd"),
    ("AA vs AKs", "Ah Ad", "Ac Kc"),
    ("KK vs AKs", "Kh Kd", "Ac Kc"),
    ("JJ vs AKo", "Jh Jd", "Ac Kd"),
    ("AKs vs QQ", "Ah Kh", "Qc Qd"),
    ("AKo vs 22", "Ah Kd", "2c 2s"),
    ("AKs vs 72o", "Ah Kh", "7c 2d"),
]

print("翻前权益对局 (10,000 次模拟)")
print("=" * 50)

results = []
for name, hand1, hand2 in matchups:
    request = EquityRequest(
        players=[
            PlayerHand(hole_cards=tuple(parse_cards(hand1))),
            PlayerHand(hole_cards=tuple(parse_cards(hand2))),
        ],
        num_simulations=10000,
        seed=42,
    )
    result = calculate_equity(request)
    eq1 = result.players[0].equity
    eq2 = result.players[1].equity
    results.append((name, eq1, eq2))
    print(f"{name:15} | {hand1:8} {eq1:5.1%} vs {eq2:5.1%} {hand2:8}")

## 权益柱状图

In [ ]:
# 创建柱状图
fig, ax = plt.subplots(figsize=(10, 6))

names = [r[0] for r in results]
eq1 = [r[1] * 100 for r in results]
eq2 = [r[2] * 100 for r in results]

x = range(len(names))
width = 0.35

bars1 = ax.bar([i - width/2 for i in x], eq1, width, label='手牌 1', color='steelblue')
bars2 = ax.bar([i + width/2 for i in x], eq2, width, label='手牌 2', color='coral')

ax.set_ylabel('权益 (%)')
ax.set_title('翻前权益对局')
ax.set_xticks(x)
ax.set_xticklabels(names, rotation=45, ha='right')
ax.legend()
ax.set_ylim(0, 100)
ax.axhline(y=50, color='gray', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()

## 收敛分析

观察随着模拟次数增加，权益估算如何收敛。

In [ ]:
# 运行权益计算并跟踪收敛过程
request = EquityRequest(
    players=[
        PlayerHand(hole_cards=tuple(parse_cards("Ah Ad"))),
        PlayerHand(hole_cards=tuple(parse_cards("Kh Kd"))),
    ],
    num_simulations=10000,
    seed=42,
    track_convergence=True,
    convergence_interval=100,
)

result = calculate_equity(request)

# 绘制收敛曲线
if result.convergence:
    sims = [p.simulation for p in result.convergence]
    eq_aa = [p.equities[0] * 100 for p in result.convergence]
    eq_kk = [p.equities[1] * 100 for p in result.convergence]
    
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.plot(sims, eq_aa, label='AA', color='steelblue', linewidth=2)
    ax.plot(sims, eq_kk, label='KK', color='coral', linewidth=2)
    
    # 添加"真实值"参考线
    ax.axhline(y=82, color='steelblue', linestyle='--', alpha=0.5)
    ax.axhline(y=18, color='coral', linestyle='--', alpha=0.5)
    
    ax.set_xlabel('模拟次数')
    ax.set_ylabel('权益 (%)')
    ax.set_title('AA vs KK: 权益收敛曲线')
    ax.legend()
    ax.set_ylim(0, 100)
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n最终权益: AA={eq_aa[-1]:.1f}%, KK={eq_kk[-1]:.1f}%")

## 三人底池

In [ ]:
# 翻前三人全押
request = EquityRequest(
    players=[
        PlayerHand(hole_cards=tuple(parse_cards("Ah Ad"))),
        PlayerHand(hole_cards=tuple(parse_cards("Kh Kd"))),
        PlayerHand(hole_cards=tuple(parse_cards("Qh Qd"))),
    ],
    num_simulations=10000,
    seed=42,
)

result = calculate_equity(request)

print("三人对局: AA vs KK vs QQ")
print("-" * 30)
hands = ["AA", "KK", "QQ"]
for i, p in enumerate(result.players):
    print(f"  {hands[i]}: {p.equity:.1%} 权益 (胜率: {p.win_rate:.1%}, 平局: {p.tie_rate:.1%})")

## 口袋对子 vs 两张高牌

经典的"抛硬币"场景。

In [ ]:
# 不同对子 vs 高牌的对局
pair_vs_overcards = [
    ("22 vs AKo", "2h 2d", "Ac Kd"),
    ("55 vs AKo", "5h 5d", "Ac Kd"),
    ("88 vs AKo", "8h 8d", "Ac Kd"),
    ("TT vs AKo", "Th Td", "Ac Kd"),
    ("JJ vs AKo", "Jh Jd", "Ac Kd"),
    ("QQ vs AKo", "Qh Qd", "Ac Kd"),
]

print("口袋对子 vs AKo")
print("=" * 40)

for name, pair, overcards in pair_vs_overcards:
    request = EquityRequest(
        players=[
            PlayerHand(hole_cards=tuple(parse_cards(pair))),
            PlayerHand(hole_cards=tuple(parse_cards(overcards))),
        ],
        num_simulations=10000,
        seed=42,
    )
    result = calculate_equity(request)
    print(f"{name:12} | 对子: {result.players[0].equity:.1%} | AKo: {result.players[1].equity:.1%}")